In [1]:
#!/usr/bin/env python
# coding: utf-8

import os, sys
import numpy as np

import xarray as xr
import pandas as pd

from dask.distributed import Client, LocalCluster
from dask.diagnostics import ProgressBar
#
# Initialisation d'un cluster de 32 coeurs
cluster = LocalCluster(processes=False, n_workers=1, threads_per_worker=8, protocol = 'tcp')
client = Client(cluster)
client

/home/durandy/miniconda3/envs/towel/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37463 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:37463/status,
Dashboard: http://127.0.0.1:37463/status,Workers: 1
Total threads: 8,Total memory: 78.61 GiB
Status: running,Using processes: False
Comm: tcp://127.0.0.1:38347,Workers: 1
Dashboard: http://127.0.0.1:37463/status,Total threads: 8
Started: Just now,Total memory: 78.61 GiB
Comm: tcp://127.0.0.1:40719,Total threads: 8
Dashboard: http://127.0.0.1:48653/status,Memory: 78.61 GiB
Nanny: None,


In [2]:
def isLeapYear(yearN):
    if ((yearN % 4 == 0) and (yearN % 100 != 0)) or (yearN % 400 == 0):
        reponse = True
    else:
        reponse = False
    return reponse

def witchYear(y,ds,ds_clim) :
    if y == 2000 :
        _ds_clim = ds_clim.isel(time = slice((31+29)*8, None))
        _ds_clim['time'] = ds['time']          
    elif isLeapYear(ds.time.dt.year.values[0]) == False :
        _ds_clim = ds_clim.isel(time = slice(0,365*int(24/3)))
        _ds_clim['time'] = ds['time']
    else :
        _ds_clim = ds_clim
        _ds_clim['time'] = ds['time']
        
    return _ds_clim

def reconstruct_Time(ds):
    year = ds.time.dt.year.data[0]
    day_start = ds.time.dt.day.data[0]
    month_start = ds.time.dt.month.data[0]
    day_end = ds.time.dt.day.data[-1]
    month_end = ds.time.dt.month.data[-1]
    
    start = str(year)+'-'+str(month_start)+'-'+str(day_start)
    end = str(year)+'-'+str(month_end)+'-'+str(day_end)
    time = pd.date_range(start = start, end = end + ' 23:00:00', freq='3H', closed = None)
    return time

In [3]:
indir_clim = '/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/SMOTHED_CLIM/'
indir_data = '/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/RAW_CLIM/'
var = 'toa_lw_all_1h'
v = var

# ds_data = xr.open_mfdataset(indir_data+'*'+var+'*.nc', chunks = {'time' : 150}, parallel=True)
# ds_data = ds_data.isel(time = slice(0,None,3))
ds_data = xr.open_mfdataset('/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/RAW_CLIM/OLR.nc', chunks = {'time' : 150}, parallel=True)
# ds_data = ds_data.isel(time = slice(0,None,3))
ds_clim = xr.open_mfdataset(indir_clim+'clim_OLR_smooth_ERA5_2012_2016.nc', chunks = {'time' : 150})

outdir = '/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/RAW_ANOMALY/OLR/'
ds_data

<xarray.Dataset>
Dimensions:        (time: 59664, lon: 360, lat: 60)
Coordinates:
  * time           (time) datetime64[ns] 2000-03-01 ... 2020-07-31T21:00:00
  * lon            (lon) float32 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * lat            (lat) float32 -29.5 -28.5 -27.5 -26.5 ... 26.5 27.5 28.5 29.5
Data variables:
    toa_lw_all_1h  (time, lat, lon) float32 dask.array<chunksize=(150, 60, 360), meta=np.ndarray>

In [5]:
year = np.arange(2012,2017)
for y in year :
#     if y == 2013 :
#         continue  
    print(y)
    ds = ds_data.sel(time = str(y))
    _Time = reconstruct_Time(ds)
    _ds_clim = witchYear(y,ds,ds_clim)
    ds = ds[v] - _ds_clim[v]
    
    ds = ds.to_dataset(name = "OLR_ano")
    ds['time'] = _Time
    ds = ds.load()
    ds.to_netcdf(outdir + 'anom_OLR_brut_ERA5_3H_'+str(y)+'_TOUCAN.nc', unlimited_dims={'time':True}, mode = 'w')

2012
2013
2014
2015
2016
